In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from matplotlib.backends.backend_pdf import PdfPages
import statsmodels.stats.multitest as smm
import numpy as np
#from bioinfokit import analys, visuz
from matplotlib.patches import Circle
from matplotlib.patheffects import withStroke
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
import matplotlib

color_palette = sns.color_palette("colorblind")

sns.set_theme(palette=color_palette, font="Arial", font_scale=1.0, style="white")


#matplotlib.rcParams.update({'axes.labelsize': 16,'xtick.labelsize': 16,})
import os


In [2]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 25)
#pd.set_option('display.max_colwidth', 300)
#pd.set_option('format.precision', 2)




In [3]:
# load results table from R DESeq analysis (gene-level, LFC shrinkage)
results_genes=pd.read_csv("gene_resAnno_only_male_231220.csv")  
results_genes.rename(columns={"Unnamed: 0": "GENEID"}, inplace = True)

results_genes#.dropna(axis=0, subset=["SYMBOL"])
results_genes #all GENEIDs are unique

,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL
0,ENSMUSG00000000001,5868.589363,0.289843,0.095989,3.019559,0.002531,0.283673,Gnai3,Gnai3
1,ENSMUSG00000000003,0.000000,NaN,NaN,NaN,NaN,NaN,Pbsn,Pbsn
2,ENSMUSG00000000028,84.565653,0.173777,0.299257,0.580695,0.561446,0.999903,Cdc45,Cdc45
3,ENSMUSG00000000037,18.402719,-1.226782,0.659554,-1.860018,0.062883,0.999903,Scml2,Scml2
4,ENSMUSG00000000049,0.480106,0.169148,4.405003,0.038399,0.969369,0.999903,Apoh,Apoh
...,...,...,...,...,...,...,...,...,...
35677,ENSMUSG00000118670,1.963773,-1.131583,2.024629,-0.558909,0.576224,0.999903,NaN,NaN
35678,ENSMUSG00000118671,258.713465,-0.302413,0.286385,-1.055968,0.290983,0.999903,EPPK1,EPPK1
35679,ENSMUSG00000118672,0.000000,NaN,NaN,NaN,NaN,NaN,Muc4,Muc4
35680,ENSMUSG00001074846,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#formatting
data=results_genes
data["padj_rounded"] = data["padj"].map('{:.1e}'.format)
data["GENENAME_padj"] = data["GENENAME"] + "\np=" + data["padj_rounded"].astype(str) +""
data["-log10_padj"] = -np.log10(data["padj"])

In [5]:
padj_thr = 0.05
lfc_thr = 1




data_all = data.dropna(subset=["log2FoldChange", "padj", "GENENAME"])
data_all_up = data_all[(data_all["log2FoldChange"] > lfc_thr) & (data_all["padj"] < 0.05)]
data_all_down = data_all[(data_all["log2FoldChange"] < -lfc_thr) & (data_all["padj"] < 0.05)]

In [6]:
data_DE_all_sortL2FC = pd.concat([data_all_up.sort_values(by=["log2FoldChange"], ascending=False),
           data_all_down.sort_values(by=["log2FoldChange"], ascending=False)])

print(len(data_DE_all_sortL2FC))
data_DE_all_sortL2FC

21


,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
18106,ENSMUSG00000069008,86.078169,22.759723,4.328613,5.257971,1.456534e-07,3.010494e-04,Gm5537,Gm5537,3.0e-04,Gm5537\np=3.0e-04,3.521362
20089,ENSMUSG00000078746,51.504298,9.557911,1.854687,5.153383,2.558289e-07,4.326299e-04,Fam205a4,Fam205a4,4.3e-04,Fam205a4\np=4.3e-04,3.363883
20283,ENSMUSG00000079224,46.828020,9.421463,1.250331,7.535177,4.876740e-14,3.023904e-10,Gm6565,Gm6565,3.0e-10,Gm6565\np=3.0e-10,9.519432
19020,ENSMUSG00000073878,6.617288,6.599175,1.632609,4.042105,5.297349e-05,2.403446e-02,Gm13304,Gm13304,2.4e-02,Gm13304\np=2.4e-02,1.619166
10926,ENSMUSG00000038155,64.622530,3.571934,0.457696,7.804166,5.989624e-15,5.570949e-11,Gstp2,Gstp2,5.6e-11,Gstp2\np=5.6e-11,10.254071
15684,ENSMUSG00000055632,67.636318,2.268333,0.586956,3.864572,1.112840e-04,4.140210e-02,Hmcn2,Hmcn2,4.1e-02,Hmcn2\np=4.1e-02,1.382978
7695,ENSMUSG00000030270,34.724822,2.236121,0.587407,3.806764,1.407971e-04,4.558897e-02,Cpne9,Cpne9,4.6e-02,Cpne9\np=4.6e-02,1.341140
12347,ENSMUSG00000042041,158.155309,1.396490,0.367481,3.800174,1.445946e-04,4.558897e-02,2010003K11Rik,2010003K11Rik,4.6e-02,2010003K11Rik\np=4.6e-02,1.341140
5302,ENSMUSG00000026064,215.115427,1.347181,0.275742,4.885666,1.030799e-06,1.268743e-03,Ptp4a1,Ptp4a1,1.3e-03,Ptp4a1\np=1.3e-03,2.896626
3089,ENSMUSG00000021453,236.026645,1.339042,0.200733,6.670754,2.544934e-11,1.183521e-07,Gadd45g,Gadd45g,1.2e-07,Gadd45g\np=1.2e-07,6.926824


In [11]:
data_DE_all_sortL2FC.sort_values(by=["log2FoldChange"], ascending=True)[["GENENAME", 
                                                                         "log2FoldChange",
                                                                         "padj"]]\
.to_excel("data_l2fc_p_DEG_only_male_231212.xlsx")

In [12]:
genes_of_interest = ["Cacna1a",
                    "Cacna1b",
                    "Cacna1c",
                    "Cacna1d",
                    "Cacna1e",
                    "Cacna1f",
                    "Cacna1g",
                    "Cacna1h",
                    "Cacna1i",
                    "Cacna2d1",
                    "Cacna2d2",
                    "Cacnb1",
                    "Cacnb2",
                    "Cacnb3",
                    "Cacnb4"
                    ]

In [13]:
data_goi = data[data["GENENAME"].isin(genes_of_interest)]
data_goi.sort_values(by=["GENENAME"], ascending=True)

,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
9696,ENSMUSG00000034656,63.427161,-0.303305,0.551366,-0.550097,0.582253,0.999903,Cacna1a,Cacna1a,1.0e+00,Cacna1a\np=1.0e+00,0.000042
673,ENSMUSG00000004113,89.045965,-0.298631,0.348240,-0.857544,0.391144,0.999903,Cacna1b,Cacna1b,1.0e+00,Cacna1b\np=1.0e+00,0.000042
14851,ENSMUSG00000051331,3734.249876,-0.026879,0.110123,-0.244082,0.807167,0.999903,Cacna1c,Cacna1c,1.0e+00,Cacna1c\np=1.0e+00,0.000042
1674,ENSMUSG00000015968,487.065786,-0.149980,0.174893,-0.857554,0.391139,0.999903,Cacna1d,Cacna1d,1.0e+00,Cacna1d\np=1.0e+00,0.000042
672,ENSMUSG00000004110,12.958336,-0.432629,0.878110,-0.492682,0.622237,0.999903,Cacna1e,Cacna1e,1.0e+00,Cacna1e\np=1.0e+00,0.000042
8133,ENSMUSG00000031142,1.986726,1.160685,2.098211,0.553178,0.580141,0.999903,Cacna1f,Cacna1f,1.0e+00,Cacna1f\np=1.0e+00,0.000042
2768,ENSMUSG00000020866,27.214688,1.242484,0.761400,1.631841,0.102713,0.999903,Cacna1g,Cacna1g,1.0e+00,Cacna1g\np=1.0e+00,0.000042
4278,ENSMUSG00000024112,2702.579277,-1.145408,0.527447,-2.171610,0.029885,0.865165,Cacna1h,Cacna1h,8.7e-01,Cacna1h\np=8.7e-01,0.062901
3640,ENSMUSG00000022416,24.844881,0.263963,0.622828,0.423814,0.671701,0.999903,Cacna1i,Cacna1i,1.0e+00,Cacna1i\np=1.0e+00,0.000042
11696,ENSMUSG00000040118,8187.157426,0.275876,0.138337,1.994224,0.046128,0.974726,Cacna2d1,Cacna2d1,9.7e-01,Cacna2d1\np=9.7e-01,0.011117


In [14]:
kegg_list = pd.read_csv("Kegg_list_mmu.csv", 
                        usecols=['GeneID', 'PathwayID','Symbol', 'ENSEMBL', "Genename"])
kegg_aldo = kegg_list[kegg_list["PathwayID"]=="path:mmu04925"]

data_aldo_genes = kegg_aldo.merge(results_genes, how="left", left_on='ENSEMBL', right_on='GENEID', validate="1:1")
print(len(data_aldo_genes.sort_values(by=["log2FoldChange"], ascending=True)))
data_aldo_genes.sort_values(by=["log2FoldChange"], ascending=True)

102


,GeneID,PathwayID,Symbol,ENSEMBL,Genename,GENEID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,GENENAME,SYMBOL,padj_rounded,GENENAME_padj,-log10_padj
62,18752,path:mmu04925,Prkcg,ENSMUSG00000078816,"protein kinase C, gamma",ENSMUSG00000078816,9.034208,-2.021360,1.234826,-1.636960,0.101639,0.999903,Prkcg,Prkcg,1.0e+00,Prkcg\np=1.0e+00,0.000042
93,58226,path:mmu04925,Cacna1h,ENSMUSG00000024112,"calcium channel, voltage-dependent, T type, al...",ENSMUSG00000024112,2702.579277,-1.145408,0.527447,-2.171610,0.029885,0.865165,Cacna1h,Cacna1h,8.7e-01,Cacna1h\np=8.7e-01,0.062901
47,15497,path:mmu04925,Hsd3b6,ENSMUSG00000027869,"hydroxy-delta-5-steroid dehydrogenase, 3 beta-...",ENSMUSG00000027869,1521.615532,-0.831973,0.835580,-0.995683,0.319404,0.999903,Hsd3b6,Hsd3b6,1.0e+00,Hsd3b6\np=1.0e+00,0.000042
57,18227,path:mmu04925,Nr4a2,ENSMUSG00000026826,"nuclear receptor subfamily 4, group A, member 2",ENSMUSG00000026826,610.064648,-0.787776,0.995054,-0.791692,0.428541,0.999903,Nr4a2,Nr4a2,1.0e+00,Nr4a2\np=1.0e+00,0.000042
99,78284,path:mmu04925,Creb3l4,ENSMUSG00000027938,cAMP responsive element binding protein 3-like 4,ENSMUSG00000027938,10.842409,-0.731438,0.923405,-0.792110,0.428297,0.999903,Creb3l4,Creb3l4,1.0e+00,Creb3l4\np=1.0e+00,0.000042
25,12292,path:mmu04925,Cacna1s,ENSMUSG00000026407,"calcium channel, voltage-dependent, L type, al...",ENSMUSG00000026407,3.567570,-0.549726,1.613205,-0.340767,0.733279,0.999903,Cacna1s,Cacna1s,1.0e+00,Cacna1s\np=1.0e+00,0.000042
76,215303,path:mmu04925,Camk1g,ENSMUSG00000016179,calcium/calmodulin-dependent protein kinase I ...,ENSMUSG00000016179,6.651426,-0.517015,1.026188,-0.503821,0.614387,0.999903,Camk1g,Camk1g,1.0e+00,Camk1g\np=1.0e+00,0.000042
6,109305,path:mmu04925,Orai1,ENSMUSG00000049686,ORAI calcium release-activated calcium modulat...,ENSMUSG00000049686,152.625862,-0.378113,0.246149,-1.536119,0.124509,0.999903,Orai1,Orai1,1.0e+00,Orai1\np=1.0e+00,0.000042
19,11932,path:mmu04925,Atp1b2,ENSMUSG00000041329,"ATPase, Na+/K+ transporting, beta 2 polypeptide",ENSMUSG00000041329,265.263562,-0.330781,0.247780,-1.334978,0.181883,0.999903,Atp1b2,Atp1b2,1.0e+00,Atp1b2\np=1.0e+00,0.000042
67,18797,path:mmu04925,Plcb3,ENSMUSG00000024960,"phospholipase C, beta 3",ENSMUSG00000024960,1643.723950,-0.306704,0.144796,-2.118175,0.034160,0.909805,Plcb3,Plcb3,9.1e-01,Plcb3\np=9.1e-01,0.041052


In [15]:
data_aldo_genes\
.sort_values(by=["padj", "log2FoldChange"], ascending=True)\
.to_excel("kegg_aldo_mmu04925_genes_only_male_231212.xlsx",columns=["Symbol", "log2FoldChange", "padj"])